In [1]:
%run theme.ipynb

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

df = pd.read_csv('f1_winners_detailed_analysis.csv')

circuit_order = [
    'China', 'Miami', 'Bahrain', 'Sakhir', 'South Korea', 'Qatar', 'Azerbaijan',
    'Brazil', 'Eifel', 'Abu Dhabi', 'India', 'USA', 'Belgium', 'Turkey', 'Russia',
    'Styria', 'Malaysia', 'Germany', 'Austria', 'Singapore'
]

available_circuits = df['Grand_Prix'].unique()
circuit_order_filtered = [circuit for circuit in circuit_order if circuit in available_circuits]

remaining_circuits = [circuit for circuit in available_circuits if circuit not in circuit_order_filtered]
final_circuit_order = circuit_order_filtered + sorted(remaining_circuits)

data_for_chart = []

for circuit in final_circuit_order:
    circuit_data = df[df['Grand_Prix'] == circuit]
    category_counts = circuit_data['Starting_Position_Category'].value_counts()
    
    p1_count = category_counts.get('P1', 0)
    p2_count = category_counts.get('P2', 0)
    p3_count = category_counts.get('P3', 0)
    outside_count = category_counts.get('Outside Top 3', 0)
    
    data_for_chart.append({
        'Circuit': circuit,
        'P1': p1_count,
        'P2': p2_count,
        'P3': p3_count,
        'Outside Top 3': outside_count,
        'Total': p1_count + p2_count + p3_count + outside_count
    })

chart_df = pd.DataFrame(data_for_chart)

colors = {
    'P1': '#DC143C',
    'P2': '#FF6B6B',
    'P3': '#FFB3B3',
    'Outside Top 3': '#E8E8E8'
}

fig = go.Figure()

fig.add_trace(go.Bar(
    name='Pole Position',
    x=chart_df['Circuit'],
    y=chart_df['P1'],
    marker_color=colors['P1'],
    marker_line_color='white',
    marker_line_width=1,
    text=[str(val) if val > 0 else '' for val in chart_df['P1']],
    textposition='inside',
    textfont=dict(color='white', size=9, family="Inter, Arial"),
    hovertemplate='<b>%{x}</b><br><span style="color:black">Pole Position: %{y} wins</span><extra></extra>'
))

fig.add_trace(go.Bar(
    name='P2 Start',
    x=chart_df['Circuit'],
    y=chart_df['P2'],
    marker_color=colors['P2'],
    marker_line_color='white',
    marker_line_width=1,
    text=[str(val) if val > 0 else '' for val in chart_df['P2']],
    textposition='inside',
    textfont=dict(color='white', size=9, family="Inter, Arial"),
    hovertemplate='<b>%{x}</b><br><span style="color:black">P2 Start: %{y} wins</span><extra></extra>'
))

fig.add_trace(go.Bar(
    name='P3 Start',
    x=chart_df['Circuit'],
    y=chart_df['P3'],
    marker_color=colors['P3'],
    marker_line_color='white',
    marker_line_width=1,
    text=[str(val) if val > 0 else '' for val in chart_df['P3']],
    textposition='inside',
    textfont=dict(color='white', size=9, family="Inter, Arial"),
    hovertemplate='<b>%{x}</b><br><span style="color:black">P3 Start: %{y} wins</span><extra></extra>'
))

fig.add_trace(go.Bar(
    name='Outside Top 3',
    x=chart_df['Circuit'],
    y=chart_df['Outside Top 3'],
    marker_color=colors['Outside Top 3'],
    marker_line_color='white',
    marker_line_width=1,
    text=[str(val) if val > 0 else '' for val in chart_df['Outside Top 3']],
    textposition='inside',
    textfont=dict(color='#666666', size=9, family="Inter, Arial"),
    hovertemplate='<b>%{x}</b><br><span style="color:black">Outside Top 3: %{y} wins</span><extra></extra>'
))

fig.update_layout(
    title=None,

    xaxis=dict(
        title=dict(
            text="Circuit",
            font=dict(size=12, color='#2C2C2C', family="Inter, Arial")
        ),
        tickfont=dict(size=9, color='#2C2C2C'),
        tickangle=-45,
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='#E0E0E0',
        mirror=False
    ),
    
    yaxis=dict(
        title=dict(
            text="Race Wins",
            font=dict(size=12, color='#2C2C2C', family="Inter, Arial")
        ),
        tickfont=dict(size=9, color='#2C2C2C'),
        showgrid=True,
        gridwidth=1,
        gridcolor='#F5F5F5',
        showline=True,
        linewidth=1,
        linecolor='#E0E0E0',
        zeroline=False
    ),
    
    barmode='stack',
    
    plot_bgcolor='white',
    paper_bgcolor='white',
    
    width=900,
    height=450,
    
    margin=dict(l=50, r=30, t=30, b=120),
    
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.35,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color='#2C2C2C', family="Inter, Arial"),
        bgcolor='rgba(255,255,255,0)',
        borderwidth=0
    ),
    
    font=dict(family="Inter, Arial, sans-serif", color='#2C2C2C'),
    showlegend=True,
    
    hoverlabel=dict(
        bgcolor="white",
        bordercolor="#E0E0E0",
        font_size=10,
        font_family="Inter, Arial",
        font_color="black"
    )
)

fig.update_xaxes(showticklabels=True, ticks="")
fig.update_yaxes(showticklabels=True, ticks="")

fig.show()